<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

Note - this is the light version that doesn't include COLMAP and therefore won't work work on images/videos that don't include camera transforms. It does still work with Polycam or Record3D input.

Because COLMAP is not included, setting up the notebook is quicker and doesn't require a restart.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
#@title # [OPTIONAL] Mount Google Drive { vertical-output: true }

#@markdown <h5>You may safely skip this step, but frequent users may find
#@markdown Google Drive convenient to load data and save results to.</h5>

#@markdown - If you do want to switch to Google Drive, you will need to change any relevant file paths throughout.

%cd /content/
# fixes weird permissions error with gdown
!pip install --upgrade --no-cache-dir gdown

from google.colab import drive
import time

drive.mount('/content/gdrive', force_remount=True)

In [2]:
#@title # Install tinycudann, nerfstudio, deps { vertical-output: true }

#@markdown <h5>Building takes a while, so we grab as much as we can pre-built from wheelhouse<h5>

%cd /content/

!pip install --upgrade pip

# Download wheelhouse
!gdown "https://drive.google.com/uc?id=1Edce0LYp7SISPl0mqFU-rpeXaoDtsa8L&confirm=t" 
!unzip "wheelhouse.zip" -d "./"

# Install tinycudann, nerfstudio & dependencies
!pip install --no-index --find-links=/content/wheelhouse tinycudann nerfstudio

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: /content/wheelhouse
Processing ./wheelhouse/tinycudann-1.7-cp38-cp38-linux_x86_64.whl
Processing ./wheelhouse/nerfstudio-0.1.18-py3-none-any.whl
Processing ./wheelhouse/functorch-1.13.1-py2.py3-none-any.whl
Processing ./wheelhouse/cryptography-38.0.4-cp36-abi3-manylinux_2_28_x86_64.whl
Processing ./wheelhouse/python_socketio-5.7.2-py3-none-any.whl
Processing ./wheelhouse/imageio-2.25.1-py3-none-any.whl
Processing ./wheelhouse/torchmetrics-0.11.1-py3-none-any.whl
Processing ./wheelhouse/ninja-1.11.1-py2.py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing ./wheelhouse/wandb-0.13.10-py3-none-any.whl
Processing ./wheelhouse/pymeshlab-2022.2.post2-cp38-cp38-manylinux1_x86_64.whl
Processing ./wheelhouse/tyro-0.3.38-py3-none-any.whl
Processing ./wheelhouse/jupyterlab-3.6.1-py3-none-any.whl
Processing ./wheelhouse/open3d-0.16.0-cp38-cp38-manylinux_2_27_x8

In [8]:
#@title # Downloading and Processing Data  { vertical-output: true }
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDE9C dozer' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
    
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"

else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content
--2023-03-07 20:26:40--  https://data.nerf.studio/nerfstudio/dozer.zip
Resolving data.nerf.studio (data.nerf.studio)... 34.102.68.79
Connecting to data.nerf.studio (data.nerf.studio)|34.102.68.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1347483375 (1.3G) [application/zip]
Saving to: ‘data/nerfstudio/dozer.zip’

data/nerfstudio/doz 100%[===================>]   1.25G  98.6MB/s    in 50s     

2023-03-07 20:27:30 (25.7 MB/s) - ‘data/nerfstudio/dozer.zip’ saved [1347483375/1347483375]

Data Processing Succeeded!


In [9]:
#@title # Set up and Start Viewer

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.349s


   ╭───────────────────────────────────────────────────────────────╮
   │                                                               │
   │      New major version of npm available! 6.14.8 → 9.6.0       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v9.6.0   │
   │               Run npm install -g npm to update!               │
   │                                                               │
   ╰───────────────────────────────────────────────────────────────╯

https://viewer.nerf.studio/?websocket_url=wss://blue-lands-work-34-83-40-38.loca.lt
You may need to click Refresh Page after you start training!


In [ ]:
#@title # Start Training  { vertical-output: true }

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    
    !ns-train nerfacto \
      --viewer.websocket-port 7007 \
      nerfstudio-data \
      --data data/nerfstudio/$scene \
      --downscale-factor 4

else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

Streaming output truncated to the last 5000 lines.
Viewer at: https://viewer.nerf.studio/versions/23-02-3-0/?websocket_url=ws://localhost:7007              
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec     Vis Rays / Sec        
-------------------------------------------------------------------------------------------------------- 
4190 (13.97%)       59.912 ms            25 m, 46 s           70.23 K                                    
4200 (14.00%)       60.664 ms            26 m, 5 s            68.97 K                                    
4210 (14.03%)       60.226 ms            25 m, 53 s           69.12 K                                    
4220 (14.07%)       62.186 ms            26 m, 43 s           68.04 K                                    
4230 (14.10%)       61.320 ms            26 m, 20 s           70.29 K                                    
4240 (14.13%)       56.733 ms            24 m, 21 s           74.50 K                                

In [ ]:
#@title # Render Video { vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/data-nerfstudio-" + scene + "/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the camera path JSON.</h3>'))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render --load-config $config_filename --traj filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4